# Introduction to `pyomo`

:::{note}
This material is in part adapted from the following resources:
- [ND Pyomo Cookbook](https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.00-Getting-Started-with-Pyomo.html)
- [Pyomo - Optimization Modeling in Python, Third Edition, Springer, 2021](https://link.springer.com/book/10.1007/978-3-030-68928-5)
- [PyPSA simple electricity market examples](https://pypsa.readthedocs.io/en/latest/examples/simple-electricity-market-examples.html)
:::

<img src="https://pyomo.readthedocs.io/en/stable/_images/PyomoNewBlue3.png" width="300px" />

[Pyomo](https://pyomo.org) is an open-source framework for formulating, solving and analysing optimisation problems with Python.

One can embed within Python an optimization model consisting of **decision variables**, **constraints**, and an optimisation **objective**, and solve these instances using commercial and open-source solvers (specialised software).

[Pyomo](https://pyomo.org) supports a wide range of problem types, including:

- **Linear programming**
- Quadratic programming
- Nonlinear programming
- Mixed-integer linear programming
- Mixed-integer quadratic programming
- Mixed-integer nonlinear programming
- Stochastic programming
- Generalized disjunctive programming
- Differential algebraic equations
- Bilevel programming
- Mathematical programs with equilibrium constraints


:::{note}
Documentation for this package is available at http://www.pyomo.org/.
:::

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install pandas pyomo highspy
```
:::

## Basic Components

Almost all Pyomo objects exist within the `pyomo.environ` namespace:

In [ ]:
import pyomo.environ as pe

A model instance is created by calling the `pe.ConcreteModel()` constructor and and assigninig it to a local variable. It's good practice to choose a short variable name to reduce the verbosity of the code.

In [ ]:
m = pe.ConcreteModel()

In [ ]:
type(m)

:::{note}
Pyomo distinguishes between *concrete* and *abstract* models. The difference is that *concrete* models have real-valued input parameters in the building stage, whereas *abstract* models are initially build with placeholders for the input parameters.
:::

### Decision Variables

**Variables** are the unknowns of an optimisation problems and are intended to be given values by solving an optimisation problem.

In `pyomo`, decision variables are created with the `pe.Var()` constructor.

In the code below, we create two decision variables:

$$x, y \in \mathbb{R}^+$$

- The name you assign the object to (e.g. `m.x`) must be unique in any given model.
- The `pe.Var()` constructor can take a variety of keyword arguments.
- For instance, `within=...` would set the variable domain.
- There are several pre-defined domains, like `pe.NonNegativeReals`.
- But you can also manually define the `bounds=...` with a tuple representing lower and upper bounds.

In [ ]:
m.x = pe.Var(within=pe.NonNegativeReals)

In [ ]:
m.y = pe.Var(bounds=(0, None))

### Objectives

An **objective** is a function of variables that returns a value that an optimization package attempts to maximize or minimize.

In `pyomo`, objectives are created with the `pe.Objective()` constructor.

In the code below, we create a simple objective function:

$$\max_{x,y} 4x + 3y$$

- Like for variables, the object returned by `pe.Objective()` is assigned to the model under a unique name.
- The keyword argument `expr=...` can be a mathematical expression or a function that returns an expression.
- The keyword argument `sense=...` determines whether the objective should be maximised or minimised. You can conveniently access the optimisation sense through `pe.minimize` and `pe.maximize`. The default is `pe.minimise`.

In [ ]:
m.obj = pe.Objective(expr=4 * m.x + 3 * m.y, sense=pe.maximize)

You can inspect almost all `pyomo` components with the `.pprint()` function:

In [ ]:
m.obj.pprint()

### Constraints

**Constraints** are equality or inequality expressions that constrain the *feasible* space of the decision variables.

In `pyomo`, constraints are created with the `pe.Constraint()` constructor.

In the code below, we create three simple constraints in different ways:

\begin{align}
  x & \leq 4 \\
  2x + y & \leq 10 \\
  x + y & \leq 10 \\
\end{align}

- Again, the object return by `pe.Constraint()` is assigned to the model under a unique name.
- Like for objectives, constraints take a mathematical expression under `expr=...`.
- Expressions can also be represented by a 3-tuple consisting of lower bound, body, upper bound.

In [ ]:
m.c1 = pe.Constraint(expr=m.x <= 4)

In [ ]:
m.c2 = pe.Constraint(expr=2 * m.x + m.y <= 10)

In [ ]:
m.c3 = pe.Constraint(expr=(None, m.x + m.y, 10))

We can also call `.pprint()` on the whole model `m` to get a full representation of the optimisation problem built:

In [ ]:
m.pprint()

## Solving optimisation problems

### Solver installation

Solvers are needed to compute solutions to the optimization models developed using Pyomo.

There exists a large variety of solvers. In many cases, they specialise in certain problem types or solving algorithms, e.g. linear or nonlinear problems.

- **open-source examples**: [CBC](https://www.coin-or.org/Cbc/), [GLPK](https://www.gnu.org/software/glpk/), [Ipopt](https://coin-or.github.io/Ipopt/), [HiGHS](https://highs.dev)
- **commercial examples**: [Gurobi](https://www.gurobi.com/), [CPLEX](https://www.ibm.com/de-de/analytics/cplex-optimizer), [FICO Xpress](https://www.fico.com/en/products/fico-xpress-optimization)

The open-source solvers are sufficient to handle meaningful Pyomo models with hundreds to several thousand variables and constraints. However, as applications get large or more complex, there may be a need to turn to a commercial solvers (which often provide free academic licenses).

Pyomo has support for a wide variety of solvers, but they need to be installed separately!

For this course, we use HiGHS, which can be installed easily via `pip` and is already in the course environment:

```sh
pip install highspy
```

If you want to try the other solvers, follow the links to their documentation for installation instructions.

The open-source solvers are sufficient to handle meaningful Pyomo models with hundreds to several thousand variables and constraints. However, as applications get large or more complex, there may be a need to turn to a commercial solvers (with free academic licenses).

:::{note}
For most solvers, it is possible to pass additional parameters before running the solver to configure things like solver tolerances, number of threads, type of solving algorithm (e.g. barrier or simplex). Read more about it in the [Pyomo Documentation](https://pyomo.readthedocs.io/en/stable/working_models.html#sending-options-to-the-solver).
:::

### Calling solvers from `pyomo`

First, we create an instance of a solver (aka optimiser):

In [ ]:
opt = pe.SolverFactory("appsi_highs")  # try also 'cbc', 'glpk', 'gurobi'

We can tell this optimiser to solve the model `m`:

In [ ]:
log = opt.solve(m, tee=True)

The keyword argument `tee=True` is optional causes a more verbose display of solver log outputs.

And then display the solver outputs:

In [ ]:
log.write()

While we can read from the message above that our problem was solved successfully, we can also formally check by accessing the reported status and comparing it to a pre-defined range of status codes provided in `pe.SolverStatus`:

In [ ]:
assert log.solver.status == pe.SolverStatus.ok

Other states of solved optimisation models are:

In [ ]:
pe.SolverStatus._member_names_

## Retrieving optimisation results

### Primal Values

Objective value:

In [ ]:
m.obj()

Decision variables

In [ ]:
m.x()

In [ ]:
m.y()

### Dual values (aka shadow prices)

Access to dual values in scripts is similar to accessing primal variable values, except that dual values are not captured by default so additional directives are needed **before** optimization to signal that duals are desired.

To signal that duals are desired, declare a **Suffix** (another `pyomo` component) with the name "dual" on the model.

In [ ]:
m.dual = pe.Suffix(direction=pe.Suffix.IMPORT)

Resolve:

In [ ]:
pe.SolverFactory("appsi_highs").solve(m).write()

Then, we can access the shadow prices of individual constraints:

In [ ]:
m.dual[m.c1]

In [ ]:
m.dual[m.c2]

## Indexed Components and Sets

Let's first create a fresh model instance:

In [ ]:
m = pe.ConcreteModel()

Almost all Pyomo components can be indexed.

While constructing them, all non-keyword arguments are assumed to represent indices.

The indices can be any iterable object.

### Sets and Range Sets

In many cases, it will be useful to make use of another basic `pyomo` component: the **set**, which is constructed by `pe.Set()` or `pe.RangeSet()`. These objects are handled very similarly to variables, constraints and objectives.

The `pe.RangeSet` will create a numbered index, starting from the value 1.

In [ ]:
m.A = pe.RangeSet(2)

In [ ]:
m.A.pprint()

With `pe.Set()` we can also pass more general indices:

In [ ]:
B = ["wind", "solar"]
m.B = pe.Set(initialize=B)

In [ ]:
m.B.pprint()

### Indexed Variables

We can then use these sets to create `pyomo` components, like variables:

In [ ]:
m.x = pe.Var(m.A)

In [ ]:
m.x.pprint()

There are no restrictions as to how many dimensions an indexed variable can have:

In [ ]:
m.y = pe.Var(m.A, m.B)

In [ ]:
m.y.pprint()

It is not strictly necessary to use *set* objects to create indexed variables. We can use any iterable object.

In [ ]:
m.z = pe.Var(B)

### List comprehension on indexed variables

List comprehensions are quite useful to process indexed variables in a compact format:

In [ ]:
m.c1 = pe.Constraint(expr=sum(m.z[b] for b in m.B) <= 10)

In [ ]:
m.c1.pprint()

### Indexed Constraints

Pyomo used the concept of *construction rules* to specify indexed constraints, rather than `expr=...`.

When building indexed constraints, particular indices are passed as argument to a rule (function) that returns an expression for each index.

In [ ]:
def construction_rule(m, a):
    return sum(m.y[a, b] for b in m.B) == 1

In [ ]:
m.c2 = pe.Constraint(m.A, rule=construction_rule)

In [ ]:
m.c2.pprint()

You can access individual constraints of a set of indexed constraints like this:

In [ ]:
m.c2[2].pprint()

It is also possible to use an alternative decorator notation to reduce redundancy in rule definitions.

In [ ]:
@m.Constraint(m.A)
def c3(m, a):
    return sum(m.y[a, b] for b in B) >= 1

## Electricity Market Examples

### Single bidding zone, single period

We want to minimise operational cost of an example electricity system representing South Africa subject to generator limits and meeting the load:

\begin{equation}
    \min_{g_s} \sum_s o_s g_s
  \end{equation}
  such that
  \begin{align}
    g_s &\leq G_s \\
    g_s &\geq 0 \\
    \sum_s g_s &= d
  \end{align}

We are given the following information on the South African electricity system:

Marginal costs in EUR/MWh

In [ ]:
marginal_costs = {
    "Wind": 0,
    "Coal": 30,
    "Gas": 60,
    "Oil": 80,
}

Power plant capacities in MW

In [ ]:
capacities = {"Coal": 35000, "Wind": 3000, "Gas": 8000, "Oil": 2000}

Inelastic demand in MW

In [ ]:
load = 42000

First step is to initialise a new model, and since we want to know about the shadow prices, we additionally pass the respective Suffix.

In [ ]:
m = pe.ConcreteModel()
m.dual = pe.Suffix(direction=pe.Suffix.IMPORT)

We create a set $S$ for the technologies:

In [ ]:
m.S = pe.Set(initialize=capacities.keys())

And the decision variables for the generator dispatch:

In [ ]:
m.g = pe.Var(m.S, within=pe.NonNegativeReals)

Our objective is to minimise total operational costs:

In [ ]:
m.cost = pe.Objective(expr=sum(marginal_costs[s] * m.g[s] for s in m.S))

In [ ]:
m.cost.pprint()

subject to the capacity limits of the generators

In [ ]:
@m.Constraint(m.S)
def generator_limit(m, s):
    return m.g[s] <= capacities[s]

In [ ]:
m.generator_limit.pprint()

and meeting energy demand

In [ ]:
m.energy_balance = pe.Constraint(expr=sum(m.g[s] for s in m.S) == load)

In [ ]:
m.energy_balance.pprint()

Then, we solve the problem:

In [ ]:
pe.SolverFactory("appsi_highs").solve(m).write()

Hopefully, the optimisation was successful (check!).

We can use `pandas` to retrieve and process the optimisation results.

In [ ]:
import pandas as pd

This is the optimal generator dispatch (in MW):

In [ ]:
pd.Series(m.g.get_values())

And the market clearing price we can read from the shadow price of the energy balance constraint (i.e. the added cost of increasing electricity demand by one unit):

In [ ]:
market_price = m.dual[m.energy_balance]
market_price

There are further interesting shadow prices. The dual values of the generator limits tell us by how much the objective would be reduced if the capacity of the respective generator would be increased. It is either zero (cf. complementary slackness) for the generators for which their capacity limit is not binding (i.e. oil and gas), or, if the constraint is binding, it denotes the difference between the market clearing price and the marginal cost of the respective generator.

Retrieve all shadow prices at once:

In [ ]:
pd.Series(m.dual.values(), m.dual.keys())

A more targeted alternative for the generator limits:

In [ ]:
pd.Series({s: m.dual[m.generator_limit[s]] for s in m.S})

### Two bidding zones with transmission

Let's add a spatial dimension, such that the optimisation problem is expanded to
\begin{equation}
  \min_{g_{i,s}, f_\ell} \sum_s o_{i,s} g_{i,s}
\end{equation}
such that
\begin{align}
  g_{i,s} &\leq G_{i,s} \\
  g_{i,s} &\geq 0 \\
  \sum_s g_{i,s} - \sum_\ell K_{i\ell} f_\ell &= d_i & \text{KCL} \\
  |f_\ell| &\leq F_\ell & \text{line limits}  \\
  \sum_\ell C_{\ell c} x_\ell f_\ell &= 0 & \text{KVL} 
\end{align}

In this example, we connect the previous South African electricity system with a hydro generation unit in Mozambique through a single transmission line. Note that because a single transmission line will not result in any cycles, we can neglect KVL in this case.

We are given the following data (all in MW):

In [ ]:
capacities = {
    "South Africa": {"Coal": 35000, "Wind": 3000, "Gas": 8000, "Oil": 2000},
    "Mozambique": {"Hydro": 1200},
}

In [ ]:
transmission = 500

In [ ]:
loads = {"South Africa": 42000, "Mozambique": 650}

In [ ]:
marginal_costs["Hydro"] = 0  # MWh

First, let's create a new model instance:

In [ ]:
m = pe.ConcreteModel()
m.dual = pe.Suffix(direction=pe.Suffix.IMPORT)

We now have two sets: one for the countries, one for the technologies, based on which we can create the generator dispatch variables.

In [ ]:
m.countries = pe.Set(initialize=loads.keys())

In [ ]:
technologies = list(capacities["South Africa"].keys() | capacities["Mozambique"])

In [ ]:
m.technologies = pe.Set(initialize=technologies)

In [ ]:
m.g = pe.Var(m.countries, m.technologies, within=pe.NonNegativeReals)

:::{note}
Note that we could have been more efficient by only creating variables for the combinations of country and technology that do exist, rather than creating variables for all combinations.
:::

We als need an additional variable for the flow.

In [ ]:
m.f = pe.Var()

The objective remains unchanged:

In [ ]:
m.cost = pe.Objective(
    expr=sum(marginal_costs[s] * m.g[c, s] for s in m.technologies for c in m.countries)
)

Generator limits also remain quite similar albeit for the additional index.
For missing entries, we set the upper limit to zero (e.g. this country-technology combination can not produce electricity).

In [ ]:
@m.Constraint(m.countries, m.technologies)
def generator_limit(m, c, s):
    return m.g[c, s] <= capacities[c].get(s, 0)

In [ ]:
m.generator_limit.pprint()

The energy balance constraint is replaced by KCL, where we take into account local generation as well as incoming or outgoing flows.

We also need the incidence matrix of this network (here it's very simple!) and assume some direction for the flow variable. Here, we picked the orientation from South Africa to Mozambique. This means that if the values for the flow variable `m.f` are positive South Africa exports to Mozambique and vice versa if the variable takes negative values.

In [ ]:
@m.Constraint(m.countries)
def kcl(m, c):
    sign = -1 if c == "Mozambique" else 1  # minimal incidence matrix
    return sum(m.g[c, s] for s in m.technologies) - sign * m.f == loads[c]

In [ ]:
m.kcl.pprint()

We also need to constrain the transmission flows to the line's rated capacity. Here, we use a 3-tuple to specify upper and lower bounds in one go.

In [ ]:
m.line_limit = pe.Constraint(expr=(-transmission, m.f, transmission))

In [ ]:
m.line_limit.pprint()

Then, we can solve and inspect results:

In [ ]:
pe.SolverFactory("appsi_highs").solve(m).write()

In [ ]:
m.cost()

In [ ]:
pd.Series(m.g.get_values()).unstack()

In [ ]:
m.f()

In [ ]:
pd.Series(m.dual.values(), m.dual.keys())

In `pyomo` it is also possible to deactivate (and activate) constraints, and fix variables to a pre-defined value, which alters the results

In [ ]:
m.line_limit.deactivate()
m.g["South Africa", "Coal"].fix(34000)

In [ ]:
pe.SolverFactory("appsi_highs").solve(m).write()

In [ ]:
pd.Series(m.g.get_values()).unstack()

In [ ]:
m.cost()

### Single bidding zone with several periods

In this example, we consider multiple time periods (labelled [1,2,3,4]) to represent variable wind generation and changing load.

  \begin{equation}
    \min_{g_{s,t}} \sum_s o_{s} g_{s,t}
  \end{equation}
  such that
  \begin{align}
    g_{s,t} &\leq \hat{g}_{s,t} G_{i,s} \\
    g_{s,t} &\geq 0 \\
    \sum_s g_{s,t} &= d_t
  \end{align}

In [ ]:
wind_ts = [0.3, 0.6, 0.4, 0.5]
load_ts = [42000, 43000, 45000, 46000]

In [ ]:
m = pe.ConcreteModel()
m.dual = pe.Suffix(direction=pe.Suffix.IMPORT)

In [ ]:
techs = capacities["South Africa"].keys()

m.S = pe.Set(initialize=techs)
m.T = pe.RangeSet(4)

In [ ]:
m.g = pe.Var(m.S, m.T, within=pe.NonNegativeReals)

In [ ]:
m.cost = pe.Objective(expr=sum(marginal_costs[s] * m.g[s, t] for s in m.S for t in m.T))

In [ ]:
@m.Constraint(m.S, m.T)
def generator_limit(m, s, t):
    cf = wind_ts[t - 1] if s == "Wind" else 1
    return m.g[s, t] <= cf * capacities["South Africa"][s]

In [ ]:
m.generator_limit.pprint()

In [ ]:
@m.Constraint(m.T)
def energy_balance(m, t):
    return sum(m.g[s, t] for s in m.S) == load_ts[t - 1]

In [ ]:
m.energy_balance.pprint()

Now, we can solve. Let's also time how long it takes to solve this problem with a small IPython tool.

In [ ]:
%%timeit -n 1 -r 1
pe.SolverFactory("appsi_highs").solve(m).write()

In [ ]:
m.cost()

In [ ]:
pd.Series(m.dual.values(), m.dual.keys())

In [ ]:
pd.Series(m.g.get_values()).unstack(0)

### Single bidding zone with several periods and storage

Now, we want to expand the optimisation model with a storage unit to do price arbitrage to reduce oil consumption.

We have been given the following characteristics of the storage:

In [ ]:
storage_energy = 6000  # MWh
storage_power = 1000  # MW
efficiency = 0.9  # discharge = charge
standing_loss = 0.00001  # per hour

To model a storage unit, we need three additional variables for the discharging and charging of the storage unit and for its state of charge (energy filling level). We can directly define the bounds of these variables in the variable definition:

In [ ]:
m.discharge = pe.Var(m.T, bounds=(0, storage_power))
m.charge = pe.Var(m.T, bounds=(0, storage_power))
m.soc = pe.Var(m.T, bounds=(0, storage_energy))

Then, we implement the storage consistency equations,

$$e_{t} = (1-\text{standing loss}) \cdot e_{t-1} + \eta \cdot g_{charge, t} - \frac{1}{\eta} \cdot g_{discharge, t}$$

For the initial period, we set the state of charge to zero.

$$e_{0} = 0$$

In [ ]:
@m.Constraint(m.T)
def storage_consistency(m, t):
    if t == 1:
        return m.soc[t] == 0
    return (
        m.soc[t]
        == (1 - standing_loss) * m.soc[t - 1]
        + efficiency * m.charge[t]
        - 1 / efficiency * m.discharge[t]
    )

In [ ]:
m.storage_consistency.pprint()

And we also need to modify the energy balance to include the contributions of storage discharging and charging.

For that, we should first remove the existing energy balance constraint, which we seek to overwrite.

In [ ]:
m.del_component(m.energy_balance)

In [ ]:
@m.Constraint(m.T)
def energy_balance(m, t):
    return sum(m.g[s, t] for s in m.S) + m.discharge[t] - m.charge[t] == load_ts[t - 1]

In [ ]:
pe.SolverFactory("appsi_highs").solve(m).write()

In [ ]:
m.cost()

In [ ]:
pd.Series(m.g.get_values()).unstack(0)

In [ ]:
pd.Series(m.charge.get_values())

In [ ]:
pd.Series(m.discharge.get_values())

In [ ]:
pd.Series(m.soc.get_values())

In [ ]:
pd.Series(m.dual.values(), m.dual.keys())

### Exercise

- Using the conversion efficiencies and specific emissions from the lecture slides, add a constraint that limits the total emissions in the four periods to 50% of the unconstrained optimal solution. How does the optimal objective value and the generator dispatch change?

- Reimplement the storage consistency constraint such that the initial state of charge is not zero but corresponds to the state of charge in the final period of the optimisation horizon.

- What parameters of the storage unit would have to be changed to reduce the objective? What's the sensitivity?